In [5]:
import pandas as pd
import typing
import numpy as np
ames=pd.read_csv('AmesHousing.csv')
ames.describe()

,Order,PID,MS SubClass,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,SalePrice
count,2930.00000,2.930000e+03,2930.000000,2440.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2907.000000,...,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000
mean,1465.50000,7.144645e+08,57.387372,69.224590,10147.921843,6.094881,5.563140,1971.356314,1984.266553,101.896801,...,93.751877,47.533447,23.011604,2.592491,16.002048,2.243345,50.635154,6.216041,2007.790444,180796.060068
std,845.96247,1.887308e+08,42.638025,23.365335,7880.017759,1.411026,1.111537,30.245361,20.860286,179.112611,...,126.361562,67.483400,64.139059,25.141331,56.087370,35.597181,566.344288,2.714492,1.316613,79886.692357
min,1.00000,5.263011e+08,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,12789.000000
25%,733.25000,5.284770e+08,20.000000,58.000000,7440.250000,5.000000,5.000000,1954.000000,1965.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000,129500.000000
50%,1465.50000,5.354536e+08,50.000000,68.000000,9436.500000,6.000000,5.000000,1973.000000,1993.000000,0.000000,...,0.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,160000.000000
75%,2197.75000,9.071811e+08,70.000000,80.000000,11555.250000,7.000000,6.000000,2001.000000,2004.000000,164.000000,...,168.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,213500.000000
max,2930.00000,1.007100e+09,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,...,1424.000000,742.000000,1012.000000,508.000000,576.000000,800.000000,17000.000000,12.000000,2010.000000,755000.000000


In [6]:
# - Cleanup
# - remove "magic literals"
# - remove lambdas
# - add types
# - Add docstring



def size_name(df_, name=''):
    print(f'{name} {df_.shape}')
    return df_

def tweak_ames(ames:pd.DataFrame) -> pd.DataFrame:
    """ Clean up raw Ames data ...
    """

    def rating_to_num(a_df:pd.DataFrame,col:str) -> pd.Series:
        return a_df[col].replace({'Ex':5, 'Gd':4, 'TA':3,'Fa':2,'Po':1})

    def create_rating_fn(col:str) -> typing.Callable:
        def wrapper(a_df):
            return rating_to_num(a_df,col)
        return wrapper

    def clean_col_name(col:str) -> str:
        return col.lower().replace(' ','_')

    def extract_story_num(a_df: pd.DataFrame) -> pd.Series:
        return a_df.house_style.str.extract(r'(\d\.?\d?)').astype(float).fillna(0)
    
    def has_basement(a_df: pd.DataFrame) ->pd.Series:
        return ~a_df.bsmt_qual.isna()

    def has_garage(a_df: pd.DataFrame) ->pd.Series:
        return ~a_df.garage_qual.isna()

    def fix_garage_year(a_df: pd.DataFrame) ->pd.Series:
        return a_df.garage_yr_blt.fillna(a_df.year_built).astype('uint8')

    def create_category_fillna_fn(col:str) ->typing.Callable:
        def wrapper(a_df: pd.DataFrame) ->pd.Series:
            return a_df[col].fillna('Other').astype('category')
        return wrapper
    
    def create_float_fillna_fn(col:str) -> typing.Callable:
        def wrapper(a_df: pd.DataFrame) -> pd.Series:
            return a_df[col].fillna(0).astype(float)
        return wrapper
    
    rating_cols=['exter_qual'  ,'exter_cond'   ,'bsmt_qual',
                  'bsmt_cond'  , 'kitchen_qual', 'garage_qual',
                  'garage_cond', 'heating_qc'  , 'fireplace_qu'
                ]

    rating_mapping = {k:create_rating_fn(k)
                        for k in rating_cols}

    category_cols = ['ms_zoning'   , 'street'        , 'alley'           , 'lot_shape'    ,   'land_contour',
                    'utilities'    , 'lot_config'    , 'land_slope'      , 'neighborhood' ,   'condition_1',
                    'condition_2'  ,  'bldg_type'    , 'house_style'     , 'roof_style'   ,   'roof_matl',
                    'exterior_1st' , 'exterior_2nd'  , 'mas_vnr_type'    , 'foundation'   ,
                    'bsmt_exposure', 'bsmtfin_type_1', 'bsmtfin_type_2'  , 'heating'      , 
                    'electrical'   , 'functional'    , 'garage_type'     , 'garage_finish',
                    'paved_drive'  , 'fence'         ,  'misc_feature'   , 'sale_type'    ,    'sale_condition']

    category_mapping = {k:create_category_fillna_fn(k) for k in category_cols}

    float_cols = ['lot_frontage' ,'mas_vnr_area'   ,'bsmtfin_sf_1'   ,'bsmtfin_sf_2'    , 
                  'bsmt_unf_sf'  ,'total_bsmt_sf'  ,'bsmt_full_bath' ,'bsmt_half_bath' ,
                  'garage_cars'  ,'garage_area']
    
    float_mapping ={k:create_float_fillna_fn(k) for k in float_cols}

    uint8_cols = ['ms_subclass'  ,'overall_qual'   ,'overall_cond' ,
                  'full_bath'    ,'half_bath'      ,'bedroom_abvgr',
                  'kitchen_abvgr', 'totrms_abvgrd' ,'fireplaces'   ,
                  'mo_sold']
    uint16_cols = ['order'          ,'year_built'   ,'year_remod/add' ,'1st_flr_sf'   ,'2nd_flr_sf' ,
                   'low_qual_fin_sf','gr_liv_area'  ,'wood_deck_sf'   ,'open_porch_sf',
                   'enclosed_porch' ,'3ssn_porch'   ,'screen_porch'   ,'pool_area'    ,'misc_val'   ,
                   'yr_sold']
    
    uint32_cols =['lot_area', 'saleprice']

    return (ames
    .pipe(size_name,'orig')
    .rename(columns=clean_col_name)
    .pipe(size_name,'rename')
    .assign(stories=extract_story_num,
            has_bsmt=has_basement,
            has_garage=has_garage,
            **rating_mapping,
            garage_yr_blt=fix_garage_year,
            **category_mapping,
            **float_mapping
             )
        .pipe(size_name,'assign')
        .astype({'central_air':bool,
            **{k:'uint8' for k in uint8_cols},
            **{k:'uint16' for k in uint16_cols},
            **{k:'uint32' for k in uint32_cols},
                })
        .pipe(size_name,'astype')
        .drop(columns=['pid'])
        .pipe(size_name,'drop')
           )


a1=tweak_ames(ames)


orig (2930, 82)
rename (2930, 82)
assign (2930, 85)
astype (2930, 85)
drop (2930, 84)


In [8]:
(a1
 .assign(**pd.DataFrame(res,columns=[f'PC{i+1}' fpr o om ramge)res.shape]))
)

,order,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,utilities,...,misc_feature,misc_val,mo_sold,yr_sold,sale_type,sale_condition,saleprice,stories,has_bsmt,has_garage
0,1,20,RL,141.0,31770,Pave,Other,IR1,Lvl,AllPub,...,Other,0,5,2010,WD,Normal,215000,1.0,True,True
1,2,20,RH,80.0,11622,Pave,Other,Reg,Lvl,AllPub,...,Other,0,6,2010,WD,Normal,105000,1.0,True,True
2,3,20,RL,81.0,14267,Pave,Other,IR1,Lvl,AllPub,...,Gar2,12500,6,2010,WD,Normal,172000,1.0,True,True
3,4,20,RL,93.0,11160,Pave,Other,Reg,Lvl,AllPub,...,Other,0,4,2010,WD,Normal,244000,1.0,True,True
4,5,60,RL,74.0,13830,Pave,Other,IR1,Lvl,AllPub,...,Other,0,3,2010,WD,Normal,189900,2.0,True,True
